In [7]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import matplotlib.dates
import urllib.request
import time
import datetime
from google.colab import files
from google.colab import data_table

In [ ]:
uploaded = files.upload()
csv_file = []
for fn in uploaded.keys():
  print('User uploaded file "{csv_file}" with length {length} bytes'.format(csv_file=fn, length=len(uploaded[fn])))
  csv_file.append(fn)

In [23]:
dados = pd.DataFrame()
locais = pd.DataFrame()
encode = 'windows-1252'
encode = 'utf-8'
for url in csv_file:
    print("Reading...",url)
    if url == 'dados.zip':
      encode = 'Latin1'
    if url == 'locais.csv':
      encode = 'utf-8'
    r = pd.read_csv(url, 
                    sep=';', 
                    index_col=False,
                    header=0,
                    encoding=encode, 
                    decimal=","
                    )
    if url == 'locais.csv':
      locais = locais.append(r)
    else:
      dados = dados.append(r)

Reading... dados.zip
Reading... locais.csv


In [ ]:
display(locais.head(1))
display(dados.head(1))

In [ ]:
display(dados.nunique())
display(dados.dtypes)

In [25]:
ndf = pd.DataFrame()
j = 0
for i in locais.index:
    secoes = locais['NÚMERO DAS SEÇÕES'][i]
    for n in secoes.split(','):
        ndf = ndf.append({'NR_SECAO':n, 'NR_ZONA':locais['ZONA'][i], 'NOM_LOCAL':locais['NOM_LOCAL'][i],'ENDEREÇO_LOCAL':locais['ENDEREÇO_LOCAL'][i], 'BAIRRO_LOCAL':locais['BAIRRO_LOCAL'][i]}, ignore_index=True)
    
ndf['NR_ZONA'] = ndf['NR_ZONA'].astype(int)
ndf['NR_SECAO'] = ndf['NR_SECAO'].astype(int)

In [ ]:
display(ndf.head(1))
display(ndf.dtypes)

In [26]:
# by merge
newDados = ndf.merge(dados, on=["NR_SECAO","NR_ZONA"], how='outer')

In [28]:
#by merge
#cargo = 'GOVERNADOR'
#cargo = 'SENADOR'
#cargo = 'DEPUTADO ESTADUAL'
cargo = 'DEPUTADO FEDERAL'
newDados = newDados.loc[newDados['DS_CARGO']==cargo].groupby(by=['NM_VOTAVEL','NR_ZONA','NR_SECAO'])

AttributeError: ignored

In [ ]:
# by merge
resultado = newDados['NM_VOTAVEL','NR_ZONA','NR_SECAO','QT_VOTOS','NOM_LOCAL','ENDEREÇO_LOCAL','BAIRRO_LOCAL']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [ ]:
# by merge
display(newDados.head(1))

In [ ]:
# by merge
display(resultado.head(1))

,NM_VOTAVEL,NR_ZONA,NR_SECAO,QT_VOTOS,NOM_LOCAL,ENDEREÇO_LOCAL,BAIRRO_LOCAL
13,MARCIO CARLOS MARINHO,154,1,5,CIEM PROFESSOR JOSELITO FALCÃO DE AMORIM,"RUA CORONEL ÁLVARO SIMÕES, S/N - ZONA URBANA",CENTRO
14,LEANDRO ALVES DIAS,154,1,1,CIEM PROFESSOR JOSELITO FALCÃO DE AMORIM,"RUA CORONEL ÁLVARO SIMÕES, S/N - ZONA URBANA",CENTRO
15,MARTA RODRIGUES SOUSA DE BRITO COSTA,154,1,1,CIEM PROFESSOR JOSELITO FALCÃO DE AMORIM,"RUA CORONEL ÁLVARO SIMÕES, S/N - ZONA URBANA",CENTRO
16,VOTO BRANCO,154,1,18,CIEM PROFESSOR JOSELITO FALCÃO DE AMORIM,"RUA CORONEL ÁLVARO SIMÕES, S/N - ZONA URBANA",CENTRO
17,CLAUDIO CAJADO SAMPAIO,154,1,1,CIEM PROFESSOR JOSELITO FALCÃO DE AMORIM,"RUA CORONEL ÁLVARO SIMÕES, S/N - ZONA URBANA",CENTRO
...,...,...,...,...,...,...,...
43434,ALEXANDRE ALELUIA DANTAS DA COSTA,154,380,1,NaN,NaN,NaN
43435,JOSÉ CERQUEIRA DE SANTANA NETO,154,380,24,NaN,NaN,NaN
43436,ELISANGELA DOS SANTOS ARAUJO,154,380,1,NaN,NaN,NaN
43437,PROGRESSISTAS,154,380,1,NaN,NaN,NaN


In [ ]:
# by brute force
dados['NOM_LOCAL'] = ""
dados['ENDEREÇO_LOCAL'] = ""
dados['BAIRRO_LOCAL'] = ""
for j in dados.index:
  for i in ndf.index:
    if dados['NR_SECAO'][j] == ndf['NR_SECAO'][i] and dados['NR_ZONA'][j] == ndf['NR_ZONA'][i]:
       dados['NOM_LOCAL'][j] = ndf['NOM_LOCAL'][i]
       dados['ENDEREÇO_LOCAL'][j] = ndf['ENDEREÇO_LOCAL'][i]
       dados['BAIRRO_LOCAL'][j] = ndf['BAIRRO_LOCAL'][i]

In [ ]:
# by brute force
dados = dados.loc[dados['DS_CARGO']=='GOVERNADOR'].groupby(by=['NM_VOTAVEL','NR_ZONA','NR_SECAO'])

In [ ]:
# by brute force
display(dados.head(1))

In [ ]:
# by brute force
resultado = dados['NM_VOTAVEL','NR_ZONA','NR_SECAO','QT_VOTOS'].sum()

In [ ]:
# by brute force
display(dados.head(1))

In [ ]:
# salvar dados em arquivo CSV
resultado.to_csv('filtragem_eleicoes2022.csv')